# Background info

### Microphysical parameters (xnames)

Note that the parameter names in the namelist correspond to the microphysics Fortran code, not the Python codes used in ensemble DA experiments. Definitions are as follows:

1. as = a_s, the coefficient in the snow fallspeed-diameter relationship
2. bs = b_s, the exponent in the snow fallspeed-diameter relationship
3. ag = a_g, the coefficient in the graupel fallspeed-diameter relationship
4. bg = b_g, the exponent in the graupel fallspeed-diameter relationship
5. tnw = N_0r, the intercept parameter in the (exponential) rain particle size distribution
6. tns = N_0s, the intercept parameter in the (exponential) snow particle size distribution
7. tng = N_0g, the intercept parameter in the (exponential) graupel particle size distribution
8. roqs = rho_s, the snow density
9. roqg = rho_g, the graupel density
10. bnd21 = q_c0, the cloud-to-rain autoconversion threshold
11. bnd1 = q_i0, the ice-to-snow autoconversion threshold (not used)

### Model output (ynames)

Running the cloud column model (CRM) will produce an ascii file (eg, crm1d_output.txt) containing a row of numbers (the default run contains 36 values in total).

If the model is run for 1 time step only, it will output 6 values corresponding to the following variables:

1. pcp: precipitation rate
2. acc: accumulated precipitation
3. lwp: liquid water path
4. iwp: ice water path
5. olr: outgoing longwave radiation
6. osr: outgoing shortwave radiation

If the model is run for "k" time steps, there will be a total of 6k values, where each set of 6 values (counting from the beginning) will correspond to the 6 variables above. E.g., the second 6-member set (starting from the 7th value and ending in the 12th value), will give the values of {pcp,acc,lwp,iwp,olr,osr} during the second model time step.

In [1]:
#----------------
#    PACKAGES    
#----------------
import time
import os,sys
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from cloud_column_model import cloud_column_model 
import create_ensemble
from parmap_framework import parmap
from module_runcrm import runcrm


#--------------------------------
#    EXPERIMENT CONFIGURATION         
#--------------------------------

### General
rand_seed = 33; np.random.seed(rand_seed) # set random number seed for reproducibility
num_Workers = 12
np.set_printoptions(precision=5) 
np.set_printoptions(suppress=True) # suppress scientific notation

### Input/output filenames
expdir = './'
input_file = expdir+'cloud_column_model/run_one_crm1d.txt' # in cloud_column_column_model dir; contains reference param vals
output_file = expdir+'cloud_column_model/crm1d_output.txt' # appears in cloud_column_model dir under the name pert_crm1d_output.txt_{num}
                                        # each column shows time evolution for 1 of the 6 model output vars 
namelist_file = expdir+'cloud_column_model/namelist_3h_t30-180.f90'

### Ensembles and distributions
LType = 1 # Gaussian likelihood by default; corresponds to additive and Gaussian noise
ens_gen = 'uniform'
n_ens = 10 # 1000 used in Posselt and Bishop (2018)
p1 = [              200.0,  0.3,  400.0,  0.4,  0.5,    0.5,   0.5,   0.2,     0.4,   1.e-3,  6.e-4 ] # mean parameter used in create_ensemble()
                                                                                                      # to define Gaussian and Gamma distributions
p2 = [              20.0,   0.05, 20.0,   0.05, 0.05,   0.05,  0.05,  0.05,    0.05,  1.e-4,  1.e-5 ] # std parameter used in create_ensemble() 
                                                                                                      # to define Gaussian and Gamma distributions
pmin = [            50.0,   0.10, 50.0,   0.1,  0.05,   0.05,  0.05,  0.05,    0.05,  1.e-4,  2.e-6 ] # lower bound
pmax = [            1000.0, 1.0,  1200.0, 0.90, 5.0,    2.5,   2.5,   1.0,     1.0,   2.e-3,  1.e-3 ] # upper bound

### Parameters and observations
# parameters
xnames = [        'as',   'bs',  'ag',  'bg', 'N0r',  'N0s', 'N0g', 'rhos',  'rhog', 'qc0', 'qi0']
XfMask = np.array([0.0,    0.0,  1.0,    1.0,  0.0,    0.0,   0.0,   0.0,     0.0,    0.0,   0.0])
x_true = [         200.0,  0.3,  400.0,  0.4,  0.5,    0.5,   0.5,   0.2,     0.4,   1.e-3,  6.e-4 ]
nx = len(xnames)
nxp = np.count_nonzero(XfMask) 
xidx = np.squeeze(np.nonzero(XfMask))
# times
tnames =          ['30', '60', '90', '120', '150', '180']
t_mask = np.array([0.0,  0.0,   0.0,  1.0,   0.0,   0.0])
nt = len(tnames)
# observations
ynames =          ['PCP', 'ACC', 'LWP', 'IWP', 'OLR', 'OSR']
y_mask = np.array([ 0.0,   0.0,   0.0,   0.0,   1.0,   1.0])
y_sigma =         [ 2.0,   5.0,   0.5,   1.0,   5.0,   5.0 ] * nt
ynames_long = [f'{y}_t{i}' for i in range(1, nt+1) for y in ynames]
n_obs = len(ynames)

### Full observation space mask
HXfMask = np.empty(n_obs * nt)
for t in range(nt):
  for o in range(n_obs):
    idx = (t * n_obs) + o
    HXfMask[idx] = t_mask[t] * y_mask[o]

In [2]:
#--------------------------------
#    MODEL SIMULATIONS         
#--------------------------------

### Generate prior ensemble X
print('Generating the prior ensemble')
p1_in   = (np.array(p1)[xidx]).tolist() # mean of selected state vars
p2_in   = (np.array(p2)[xidx]).tolist() # std of -||-
pmin_in = (np.array(pmin)[xidx]).tolist() # lower bound of -||-
pmax_in = (np.array(pmax)[xidx]).tolist() # upper bound of -||-
# only perturb selected state vars; Xf dims will be [Ne,Nx_selected]
Xf_subset = create_ensemble.create_ensemble(n_ens,p1_in,p2_in,pmin_in,pmax_in,ens_gen)
print('     Shape of Xf with selected vars:   ',np.array(Xf_subset).shape)
# now construct the full prior ensemble in which selected vars are perturbed
# and masked variables are set to their true values for all ensemble members
Xf_mask = np.tile(x_true,n_ens).reshape(n_ens,len(x_true))
Xf_mask[:,xidx] = np.array(Xf_subset)
Xf = Xf_mask.tolist() # list in the form [[parms_mem{1}],...,[parms_mem{n_ens}]]
                        # where parms_mem{k} is made of the 11 state vars (CRM params)
print('     Shape of Xf with all vars:        ',np.array(Xf).shape)
print('     Prior mean (Xf.mean):             ',np.mean(Xf,axis=0)[xidx])
print('     Prior variance (Xf.var):          ',np.var(Xf,axis=0)[xidx])

### Mapping prior ensemble to observation space h(X) (i.e., run ensemble of CRM simulations)
print('Mapping prior ensemble to observation space (running the cloud model)')
runs = [] # full input list for the ensemble CRM runs
input_file_list = [input_file] * n_ens
output_file_list = [output_file] * n_ens
namelist_file_list = [namelist_file] * n_ens
run_num_list = list(range(1,n_ens+1))
runs = [list(x) for x in zip(input_file_list, output_file_list, namelist_file_list, run_num_list, Xf)]
print(f'    Full input to first ensemble member: {runs[0]}')
DASK_URL = 'scispark6.jpl.nasa.gov:8786'
parmode = 'par'
pmap = parmap.Parmap(master=DASK_URL, mode=parmode, numWorkers=num_Workers)
HXf = pmap(runcrm, runs)
print('    State vector for ens member Ne/2:  ', Xf[np.int32(n_ens/2)])
print('    ObSpace-mapped state vector for ens member Ne/2: ', HXf[np.int32(n_ens/2)])
print('    len(HXf), len(HXf[0]): ',len(HXf),len(HXf[0]))

Generating the prior ensemble
     Shape of Xf with selected vars:    (10, 2)
     Shape of Xf with all vars:         (10, 11)
     Prior mean (Xf.mean):              [592.77307   0.50068]
     Prior variance (Xf.var):           [148735.7761       0.04058]
Mapping prior ensemble to observation space (running the cloud model)
    Full input to first ensemble member: ['./cloud_column_model/run_one_crm1d.txt', './cloud_column_model/crm1d_output.txt', './cloud_column_model/namelist_3h_t30-180.f90', 1, [200.0, 0.3, 335.7866465533894, 0.4599803368406364, 0.5, 0.5, 0.5, 0.2, 0.4, 0.001, 0.0006]]



parmap <function runcrm at 0x14dbb5120>: Running in mode par with numPartitions 12


Warning! Parameter outside allowable range
Ensemble member, parameter number, min, value, max:          1   3        75.0000000000        72.6106414795      1200.0000000000
    State vector for ens member Ne/2:   [200.0, 0.3, 1159.7808428458472, 0.4147189912973067, 0.5, 0.5, 0.5, 0.2, 0.4, 0.001, 0.0006]
    ObSpace-mapped state vector for ens member Ne/2:  [3.7471554279, 0.0863199979, 5.0852184296, 0.0008192234, 240.7518310547, 896.9567871094, 10.3959369659, 4.1528553963, 5.0704741478, 3.6567947865, 156.5317687988, 798.548034668, 7.1567687988, 8.5850858688, 3.3200466633, 10.650478363, 180.2579650879, 821.0228271484, 12.0394258499, 13.2836961746, 4.6356291771, 13.6661300659, 201.7253723145, 839.5126953125, 15.5236377716, 20.4406204224, 5.1266746521, 12.5662965775, 207.0706939697, 752.5579833984, 22.8373794556, 29.3255386353, 7.0462384224, 4.0525369644, 230.0921630859, 495.8077087402]
    len(HXf), len(HXf[0]):  10 36


In [3]:
#------------------------
#    TRUE SIMULATION            
#------------------------

### Run the cloud model with the true parameters
print('*** TRUE SIMILATION ***')
crm1d = cloud_column_model.CRM1DWrap(input_file,output_file,namelist_file, params=x_true,verbose=True)
y_true, crm_status = crm1d() # a list of len(y_true) = nt*n_obs
print('Output: ',len(y_true),y_true)

*** TRUE SIMILATION ***
Parameter values to be used in cloud_column_model.py:  200.0 0.3 400.0 0.4 0.5 0.5 0.5 0.2 0.4 0.001 0.0006
Input file name:     ./cloud_column_model/run_one_crm1d.txt-CRM1D-3406b966b8
Output file name:    ./cloud_column_model/crm1d_output.txt-CRM1D-7420fb0b34
Namelist file name:  ./cloud_column_model/namelist_3h_t30-180.f90
Output:  36 [3.7471554279, 0.0863199979, 5.0852060318, 0.0008276912, 240.7512512207, 896.9626464844, 7.9930844307, 3.937908411, 4.4381356239, 4.5186429024, 153.4401702881, 804.2377929688, 6.2555627823, 7.6948990822, 2.8587121964, 12.063126564, 160.9409637451, 848.690612793, 12.8107385635, 11.4412117004, 5.5484800339, 14.7016458511, 195.8087310791, 803.0979003906, 18.7041091919, 21.073266983, 5.6027178764, 11.48528862, 212.2489471436, 737.3928222656, 20.8552017212, 30.281162262, 6.4446291924, 3.7101278305, 231.7576751709, 493.205291748]
